In [8]:
#@title Imports
import sys
import agent.utility_classes as util
import agent.agent_classes as ac
from typing import TypeVar
import vertexai

import tenacity
from vertexai.generative_models import GenerationConfig
from vertexai.generative_models import GenerativeModel

In [9]:
# Vertex AI model ids
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
DEFAULT_LLM_VERTEX_ID = 'gemini-1.5-pro-002' 
vertexai.init(project=<your_project_id>, location="us-central1")

In [10]:
llm_client = util.LLM()

In [11]:
#@title User prompt
prompt = 'generate an image of a pig flying in the sky with an eagle' # @param
state = None

In [12]:
# Ensure the LLM client is running
llm_client.generate('generate something')

"A shimmering dragonfly, its wings like stained glass, landed on a dew-kissed blade of grass.  Sunlight, fractured through the droplets, painted fleeting rainbows on the insect's iridescent body. It paused, antennae twitching, sensing the subtle vibrations of the morning. Then, with a whir of its delicate wings, it launched itself into the air, a flash of sapphire against the emerald backdrop of the meadow.  A gentle breeze carried it towards a stand of ancient willows, their weeping branches trailing in the crystal clear stream.  The dragonfly, a tiny jewel in the vastness of nature, continued its silent dance, a fleeting moment of beauty in the unfolding day.\n"

In [13]:
agent_type = 'attribute' # @param ['belief', 'attribute', 'llm'] {isTemplate: true}
print('Initializing the agent. This can take a while...')
if agent_type == 'belief':
  agent = ac.BeliefAgent(
      prompt, llm_client, state=state
  )
elif agent_type == 'attribute':
  agent = ac.AttributeAgent(
      prompt, llm_client, state=state
  )
elif agent_type == 'llm':
  agent = ac.LLMAgent(
      prompt, llm_client, state=state
  )
else:
  raise ValueError(f'Unknown agent type: {agent_type}')
state = agent.state

Initializing the agent. This can take a while...


# Run multi-turn dialogue between user and agent

Enter "\_print_agent_state\_" to print the state of the agent.

Enter "\_print_new_prompt\_" to print the new T2I prompt that incorporates all user provided information so far.

In [14]:
print('User:', prompt)
MAX_TURN = 5
for _ in range(MAX_TURN):
  action = agent.select_action(verbose=False)
  if action is None:
    print('Done')
    break
  print('Agent:', agent.verbalize_action(action))
  observation = input('User: ')
  if observation == '_print_agent_state_':
    print(agent.state)
  elif observation == '_print_new_prompt_':
    print(agent.prompt)
    sys.stdout.flush()
  else:
    print('\nUpdating agent state... This can take a while...')
    sys.stdout.flush()
    agent.transition(action, observation)
  print('\n\n')

User: generate an image of a pig flying in the sky with an eagle
Agent: What species of eagle are you thinking of? a. Bald Eagle, b. Golden Eagle, c. Harpy Eagle, d. Philippine Eagle, e. Steller's Sea Eagle f. unkown. If none of these options, what species of eagle are you thinking of?



User:  stellar sea eagle



Updating agent state... This can take a while...



Agent: What should the relation pig-eagle be? a. in front of, b. behind, c. left of, d. right of, e. above f. unkown. If none of these options, what should the relation pig-eagle be?



User:  behind



Updating agent state... This can take a while...



Agent: What should the relation pig-eagle be? a. in front of, b. behind, c. left of, d. right of, e. above f. unkown. If none of these options, what should the relation pig-eagle be?



...


Updating agent state... This can take a while...



Agent: What should the image style be? a. cartoon, b. photorealistic, c. painting, d. digital art, e. drawing f. unkown. If none of these options, what should the image style be?



User:  digital art



Updating agent state... This can take a while...



Agent: What should the time of day be? a. daytime, b. sunset, c. sunrise d. unkown. If none of these options, what should the time of day be?



User:  sunset



Updating agent state... This can take a while...





In [15]:
print(f"Refined new prompt:")
print(f"{agent.prompt}")

Refined new prompt:
A Steller's Sea Eagle soars through the sunset sky, a pig flying behind it. The pig is behind the eagle. This scene is depicted in a digital art style.

